In [117]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import base64
from io import BytesIO
from IPython.display import Audio, display
import anthropic

In [118]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")
    
GPT_MODEL = "gpt-4o-mini"
openai = OpenAI()

ANTHROPIC_MODEL="claude-3-haiku-20240307",
claude = anthropic.Anthropic()

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-


In [119]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."
system_message += "When booking a flight always check if the desired time is really available."

translate_system_message = "You are a language assistant that translates text between English and German. "
translate_system_message += "Translate the given text from English to German accurately while preserving the meaning and tone. "
translate_system_message += "Return only the translated text without explanations."

In [120]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [121]:
flight_times = {"london": ["08:00", "11:42", "13:30", "15:47"], "paris": ["07:30", "10:44", "17:30", "22:37"], "tokyo": ["08:54", "14:59"], "berlin": ["06:52", "09:43", "13:56"]}

def get_flight_times(destination_city):
    print(f"Tool get_flight_times called for {destination_city}")
    city = destination_city.lower()
    return flight_times.get(city, "Unknown")    

In [122]:
def book_flight(destination_city, time):
    print(f"Booking flight to {destination_city} at {time}")
    return f"Flight to {destination_city.title()} booked for {time}. Confirmation sent!"

In [123]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [124]:
flight_times_function = {
    "name": "get_flight_times",
    "description": "Get the times of fights to location. Call this whenever you need to know the flight times, for example when a customer want to book a flight to a city, provide him with the time options.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [125]:
book_flight_function = {
    "name": "book_flight",
    "description": "Book a flight to a city at a specific time.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "City to fly to"
            },
            "time": {
                "type": "string",
                "description": "Departure time in HH:MM"
            }
        },
        "required": ["destination_city", "time"]
    }
}


In [126]:
tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": flight_times_function},
    {"type": "function", "function": book_flight_function}
]

In [127]:
def handle_tool_call(message):
    tool_responses = []
    for tool_call in message.tool_calls:
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Handling tool call for function: {function_name}, arguments: {arguments}")
        city = arguments.get("destination_city")

        if function_name == "get_ticket_price":
            result = get_ticket_price(city)
        elif function_name == "get_flight_times":
            result = get_flight_times(city)
        elif function_name == "book_flight":
            time = arguments.get("time")
            result = book_flight(city, time)
        else:
            result = "Unknown function"

        tool_responses.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": function_name,
            "content": json.dumps(result)
        })
    return tool_responses, city


In [136]:
def translate_to_german(text):
    response = claude.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        messages=[
            {"role": "user", "content": f"{translate_system_message}\n\nTranslate this to German:\n\n{text}"}
        ]
    )
    return response.content[0].text.strip()

In [137]:
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

In [138]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    response = openai.chat.completions.create(model=GPT_MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        assistant_message = response.choices[0].message
        tool_messages, city = handle_tool_call(assistant_message)
        messages.append(assistant_message)
        messages.extend(tool_messages)
        response = openai.chat.completions.create(model=GPT_MODEL, messages=messages)

    reply = response.choices[0].message.content

    # Add to English history
    history.append({"role": "assistant", "content": reply})

    # Translate both user and assistant messages
    german_user = translate_to_german(message)
    german_reply = translate_to_german(reply)

    # Build translated history separately
    german_history = []
    for msg in history:
        if msg["role"] == "user":
            german_msg = translate_to_german(msg["content"])
            german_history.append({"role": "user", "content": german_msg})
        elif msg["role"] == "assistant":
            german_msg = translate_to_german(msg["content"])
            german_history.append({"role": "assistant", "content": german_msg})

    # Optional: speak the assistant reply in English
    talker(reply)

    return history, german_history


In [139]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot_en = gr.Chatbot(label="FlightAI Assistant (English)", height=500, type="messages")
        chatbot_de = gr.Chatbot(label="FlightAI Assistant (German)", height=500, type="messages")

    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")

    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history_en, history_de):
        history_en += [{"role":"user", "content":message}]
        return "", history_en, history_de

    entry.submit(
        do_entry,
        inputs=[entry, chatbot_en, chatbot_de],
        outputs=[entry, chatbot_en, chatbot_de]
    ).then(
        chat,
        inputs=[entry, chatbot_en],
        outputs=[chatbot_en, chatbot_de]
    )

    clear.click(lambda: ([], []), outputs=[chatbot_en, chatbot_de], queue=False)

ui.launch(inbrowser=True)


* Running on local URL:  http://127.0.0.1:7877

To create a public link, set `share=True` in `launch()`.


Handling tool call for function: get_flight_times, arguments: {'destination_city': 'Paris'}
Tool get_flight_times called for Paris


Handling tool call for function: book_flight, arguments: {'destination_city': 'Paris', 'time': '17:30'}
Booking flight to Paris at 17:30
